##### Copyright 2020 The AdaNet Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AdaNet ModelFlow Tutorial

AdaNet ModelFlow is a framework to define end-to-end AutoML workflows. In this tutorial, we will outline a sample workflow for how to construct and run a ModelFlow pipeline.

Our pipeline will iteratively construct an ensemble of CNN architectures to classify Fashion MNIST examples.

## Imports
The necessary imports are `tensorflow` and `adanet`. We will be using `kerastuner` to generate our subnetworks.

In [0]:
!pip install adanet, kerastuner

In [0]:
# Load TensorBoard magic.
%load_ext tensorboard

import time
import kerastuner

import adanet.experimental as adanet
import numpy as np
import tensorflow.compat.v1 as tf

RANDOM_SEED = 42

ModelFlow is built on TensorFlow 2.0, so make sure to enable 2.0 behavior.

In [0]:
tf.enable_v2_behavior()

## Loading Fashion MNIST Dataset

Conveniently, the dataset can be loaded easily from within TensorFlow.

In [0]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Perform some basic preprocessing on our data and convert them to `tf.data.Dataset` objects.

In [0]:
# Normalize images.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Adding a dimension to conform to what a Keras model expects.
train_images = np.expand_dims(train_images, -1)
test_images = np.expand_dims(test_images, -1)

# Construct datasets.
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

# Note: we are only repeating the dataset once here for demo purposes.
# Increasing the repetition size would improve model performance, but it would
# also slow training.
train_dataset = train_dataset.shuffle(1000, seed=RANDOM_SEED).batch(64).repeat(1)
test_dataset = test_dataset.batch(64)

## Setup Keras Tuner Model

We will use [Keras Tuner](https://github.com/keras-team/keras-tuner) to define, train and tune our candidate networks.

In [0]:
def build_model(hp):

  # Define hyperparameter search space.
  filters = hp.Choice('filters', values=[16, 32])
  kernel_size = hp.Choice('kernel_size', values=[2, 3])
  pool_size = hp.Choice('pool_size', values=[2, 4])
  learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  dropout1 = hp.Float('dropout1', min_value=0.5, max_value=0.9, step=0.1)
  dense1_units = hp.Choice('dense1_units', values=[64, 128])
  dropout2 = hp.Float('dropout2', min_value=0.5, max_value=0.9, step=0.1)

  # Define model architecture.
  # Credit: https://github.com/abelusha/MNIST-Fashion-CNN/blob/master/Fashon_MNIST_CNN_using_Keras_10_Runs.ipynb
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Conv2D(filters, kernel_size, activation='relu'))
  model.add(tf.keras.layers.MaxPool2D())
  model.add(tf.keras.layers.Conv2D(filters, kernel_size, activation='relu'))
  model.add(tf.keras.layers.MaxPool2D())
  model.add(tf.keras.layers.Dropout(dropout1, seed=RANDOM_SEED))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(dense1_units, activation='relu'))
  model.add(tf.keras.layers.Dropout(dropout2, seed=RANDOM_SEED))
  model.add(tf.keras.layers.Dense(10, activation='softmax'))

  model.compile(
    optimizer=tf.keras.optimizers.Adam(
      learning_rate=learning_rate),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])
  
  return model

## Define our AutoML Pipeline

We define an AutoML pipeline simulating the AdaNet algorithm.

In [0]:
# Create a shared storage between AutoEnsemblePhases.
autoensemble_storage = adanet.storages.InMemoryStorage()

# Set pipeline parameters.
# Note: We use few repetitions and max_trials for demonstration
# purposes. Increasing these values should improve model
# performance, but it would also slow training.
#@title Pipeline Parameters
max_trials = 10#@param {'type': 'integer'}
repetitions = 2#@param {'type': 'integer'}

# Define pipeline phases.
input_phase = adanet.phases.InputPhase(train_dataset, test_dataset)
repeat_phase = adanet.phases.RepeatPhase(
  phase_factory=[
    lambda: adanet.phases.KerasTunerPhase(
      kerastuner.tuners.RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=max_trials,
        executions_per_trial=1,
        directory='/tmp',
        # Make sure each KerasTunerPhase has a unique project_name.
        project_name="tutorial"+str(int(time.time())),
        overwrite=True,
        seed=RANDOM_SEED,
      )
    ),
    lambda: adanet.phases.AutoEnsemblePhase(
      ensemblers=[
        adanet.phases.autoensemble_phase.MeanEnsembler(
            'sparse_categorical_crossentropy', 
            'adam', 
            ['accuracy'])
      ],
      ensemble_strategies=[
        adanet.phases.autoensemble_phase.GrowStrategy(), 
        adanet.phases.autoensemble_phase.AllStrategy(), 
      ],
      storage=autoensemble_storage,
      num_candidates=4
    )
  ],
  repetitions=repetitions
)

controller = adanet.controllers.SequentialController([input_phase, 
                                                      repeat_phase])
model_search = adanet.keras.ModelSearch(controller)

## Run our AutoML Pipeline

Single command to launch your pipeline.

In [0]:
model_search.run()

## Visualize the run in TensorBoard

Setting the y-axis to wall time best illustrates the training process.

In [0]:
%tensorboard --logdir=/tmp/ --port=0

## Visualize top models from run

We first obtain the top model and then we print its summary. We also print the subnetwork summaries to examine the structure of the top performing ensemble.

Since the resulting ensembles are just Keras models, we can use them in any context that Keras models can be used!



In [0]:
best_models = list(model_search.get_best_models(1))
best_models[0].summary()

In [0]:
for submodel in best_models[0].submodels:
  print(submodel.summary())